# Notebook imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [17]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'
VOCAB_SIZE = 2500

# Read and Load Features from .txt file

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [9]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [11]:
sparse_train_data[-5:]

array([[5795, 2033,    0,    1],
       [5795, 2036,    0,    2],
       [5795, 2278,    0,    1],
       [5795, 2339,    0,    1],
       [5795, 2357,    0,    1]])

In [13]:
print('Nr of rows in training file: ', sparse_train_data.shape[0])
print('Nr of rows in testing file: ', sparse_test_data.shape[0])

Nr of rows in training file:  258358
Nr of rows in testing file:  117782


In [16]:
print('Nr of emails in training file: ', np.unique(sparse_train_data[:, 0]).size)
print('Nr of emails in testing file: ', np.unique(sparse_test_data[:, 0]).size)

Nr of emails in training file:  4013
Nr of emails in testing file:  1724


## Creating An Empty Dataframe

In [18]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [19]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [21]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Creating A Full Matrix from a Sparse Matrix

In [24]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Forms a full matrix from a sparse matrix. Returns a pandas dataframe
    Arguments:
    sparse_matrix -- numpy array
    nr_words -- number of words in the vocabulary. Total Number of tokens
    doc_idx -- position of document ID in sparse matrix. Default = column 0
    word_idx -- position of word ID in sparse  matrix. Default = column 1
    cat_idx -- position of category(0 for non-spam, 1 for spam) in sparse matrix. Default = column 2
    freq_ix -- position of number of occurences of word in sparse matrix. Default = column 3
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [25]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 1min 32s


In [26]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training The Naive Bayes Model

## Calculating the Probability of Spam

In [29]:
nr_spam_emails = full_train_data[full_train_data.CATEGORY == 1].shape[0]

In [30]:
nr_emails = full_train_data.shape[0]

In [34]:
prob_spam = nr_spam_emails / nr_emails 
prob_spam

0.310989284824321

## Total Number of Words / Tokens

In [38]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
email_length = full_train_features.sum(axis=1)
email_length[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [46]:
total_wc = email_length.sum()
total_wc

429243

## Number of tokens in Ham & Spam Emails

In [49]:
full_train_ham = full_train_data[full_train_data.CATEGORY == 0]
full_train_ham_features = full_train_ham.loc[:, full_train_data.columns != 'CATEGORY']
full_train_ham

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,0,1,2,1,0,1,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
1898,0,2,1,0,1,1,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1899,0,2,1,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1900,0,3,4,0,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,0,2,2,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [51]:
ham_email_length = full_train_ham.sum(axis=1)
ham_email_length[:5]

DOC_ID
1896     82
1898    112
1899     66
1900     49
1901     34
dtype: int64

In [53]:
total_wc_ham = ham_email_length.sum()
total_wc_ham

252893

In [69]:
email_length.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    9,   10,
            ...
            5781, 5782, 5784, 5785, 5787, 5789, 5790, 5791, 5794, 5795],
           dtype='int64', name='DOC_ID', length=4013)

In [80]:
ham_wc = email_length[email_length.index >= 1896].sum()

In [82]:
ham_wc

252893

In [83]:
spam_wc = email_length[email_length.index <= 1895].sum()
spam_wc

176350

In [91]:
spam_len = email_length[full_train_data.CATEGORY == 1]
spam_len.shape
spam_wc = spam_len.sum()
spam_wc

176350

In [90]:
ham_len = email_length[full_train_data.CATEGORY == 0]
ham_len.shape
ham_wc = ham_len.sum()
ham_wc

252893

In [88]:
# checking if the subsetting was correct
email_length.shape[0] - spam_len.shape[0] - ham_len.shape[0]

0